# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [58]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/domenicrhedrick/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [65]:
# Read your api key environment variable
api_key = os.getenv("newsapi")

print(api_key)

None


In [ ]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)


In [ ]:
# Fetch the Bitcoin news articles
bitcoin_news = newsapi.get_everything(q= "bitcoin", language="en")


In [ ]:
# Fetch the Ethereum news articles
etherium_news = newsapi.get_everything(
    q="etherium",
    language="en"
)

In [ ]:
#Initialize the Vader Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()


In [ ]:
#Create a empty list for Bitcoin dataset

bitcoin_sentiment= []

# Loop to iterate through each row

for article in bitcoin_news["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]                                                                                                                                                                                             
        neg = sentiment["neg"]
        
        bitcoin_sentiment.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu })
    except AttributeError:
        pass


# Create the Bitcoin sentiment scores DataFrame

bitcoin_sentiment_df = pd.DataFrame(bitcoin_sentiment)

cols = ["date", "text", "compound", "positive", "negative", "neutral"]
bitcoin_sentiment_df = bitcoin_sentiment_df[cols]
bitcoin_sentiment_df

In [ ]:

#Create a empty list for Ether dataset
etherium_sentiment= []

# Loop to iterate through each row
for article in etherium_news["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]                                                                                                                                                                                             
        neg = sentiment["neg"]
        
        etherium_sentiment.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu })
    except AttributeError:
        pass



# Create the Bitcoin sentiment scores DataFrame
etherium_sentiment_df = pd.DataFrame(etherium_sentiment)

#Set Columns inside dataset
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
ehterium_sentiment_df = etherium_sentiment_df[cols]



In [ ]:
# Describe the Bitcoin Sentiment
bitcoin_sentiment_df.describe()


In [ ]:
# Describe the Ethereum Sentiment
etherium_sentiment_df.describe()


### Questions:

Q: Which coin had the highest mean positive score?

A: Ethereum (mean positive .114)

Q: Which coin had the highest compound score?

A: Ethereum with a compound score of .913

Q. Which coin had the highest positive score?

A: Ethereum with .306

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [ ]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
nltk.download('stopwords')
sw = set(stopwords.words('english'))

# Expand the default stopwords list if necessary
sw_addons = {'/prnewswire/', '--', "'s"}


In [ ]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text
    list_token = list(filter(lambda w: w not in punctuation, list_token))

   
    # Create a tokenized list of the words
    list_token = word_tokenize(text)

    
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in list_token]

   
    # Convert the words to lowercase
    tokens = [word.lower() for word in lem if word.lower() not in sw.union(sw_addons)]
    
    
    # Remove the stop words
    list_token = list(filter(lambda w: w.lower() not in sw, list_token))
    
    
    return tokens

In [ ]:
# Create a new tokens column for Bitcoin
bitcoin_sentiment_df["tokens"]= bitcoin_sentiment_df.text.apply(tokenizer)

bitcoin_sentiment_df

In [ ]:
# Create a new tokens column for Ethereum
etherium_sentiment_df["tokens"]= etherium_sentiment_df.text.apply(tokenizer)

etherium_sentiment_df

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
bitcoin_ngrams= ngrams(tokenizer(bitcoin_sentiment_df.text.str.cat()), n=2)


In [ ]:
# Generate the Ethereum N-grams where N=2
etherium_ngrams= ngrams(tokenizer(etherium_sentiment_df.text.str.cat()), n=2)


In [ ]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
# Use token_count to get the top 10 words for Bitcoin
token_count(bitcoin_ngrams, 10)


In [ ]:
# Use token_count to get the top 10 words for Ethereum
token_count(etherium_ngrams, 10)


---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
wc = WordCloud().generate(bitcoin_sentiment_df.text.str.cat())
plt.imshow(wc)

In [ ]:
# Generate the Ethereum word cloud
wc = WordCloud().generate(etherium_sentiment_df.text.str.cat())
plt.imshow(wc)

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Download the language model for SpaCy
!python -m spacy download en_core_web_sm


In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [ ]:
# Concatenate all of the Bitcoin text together
bitcoin_con = bitcoin_sentiment_df.text.str.cat()


In [ ]:
# Run the NER processor on all of the text
bitcoin_doc = nlp(bitcoin_con)

# Add a title to the document
bitcoin_doc.user_data["title"]= "Bitcoin and its NER"


In [ ]:
# Render the visualization
displacy.render(bitcoin_doc, style='ent')


In [ ]:
# List all Entities
print([ent.text for ent in bitcoin_doc.ents if ent])


---

### Ethereum NER

In [ ]:
# Concatenate all of the Ethereum text together
etherium_con = etherium_sentiment_df.text.str.cat()


In [ ]:
# Run the NER processor on all of the text
etherium_doc = nlp(etherium_con)

# Add a title to the document
etherium_doc.user_data["title"]= "Etherium and its NER"


In [ ]:
# Render the visualization
displacy.render(etherium_doc, style='ent')


In [ ]:
# List all Entities
print([ent.text for ent in etherium_doc.ents if ent])


---